In [2]:
!nvidia-smi

Thu Aug 31 13:49:10 2023       
+---------------------------------------------------------------------------------------+
| NVIDIA-SMI 535.54.03              Driver Version: 535.54.03    CUDA Version: 12.2     |
|-----------------------------------------+----------------------+----------------------+
| GPU  Name                 Persistence-M | Bus-Id        Disp.A | Volatile Uncorr. ECC |
| Fan  Temp   Perf          Pwr:Usage/Cap |         Memory-Usage | GPU-Util  Compute M. |
|                                         |                      |               MIG M. |
|=========================================+======================+======================|
|   0  Tesla T4                       Off | 00000000:AF:00.0 Off |                    0 |
| N/A   70C    P0              33W /  70W |      2MiB / 15360MiB |      7%      Default |
|                                         |                      |                  N/A |
+-----------------------------------------+----------------------+--

In [4]:
# !ls -lah /root/.cache/huggingface/hub/

In [36]:
!ls -lah /root/.cache/torch/sentence_transformers/GanymedeNil_text2vec-large-chinese

total 2.5G
drwxr-xr-x 2 root root  237 2023-08-31__10:56:08 .
drwxr-xr-x 4 root root   76 2023-08-31__10:56:08 ..
-rw-r--r-- 1 root root 1.5K 2023-08-31__09:35:24 .gitattributes
-rw-r--r-- 1 root root  443 2023-08-31__09:35:24 README.md
-rw-r--r-- 1 root root  821 2023-08-31__09:35:25 config.json
-rw-r--r-- 1 root root   69 2023-08-31__09:35:26 eval_results.txt
-rw-r--r-- 1 root root 1.3G 2023-08-31__10:15:04 model.safetensors
-rw-r--r-- 1 root root 1.3G 2023-08-31__10:56:04 pytorch_model.bin
-rw-r--r-- 1 root root  125 2023-08-31__10:56:05 special_tokens_map.json
-rw-r--r-- 1 root root 430K 2023-08-31__10:56:06 tokenizer.json
-rw-r--r-- 1 root root  514 2023-08-31__10:56:07 tokenizer_config.json
-rw-r--r-- 1 root root 107K 2023-08-31__10:56:08 vocab.txt


In [5]:
# import sys,os,os.path
# os.environ['HTTP_PROXY']="http://127.0.0.1:8098"
# os.environ['HTTPS_PROXY']="http://127.0.0.1:8098"

In [6]:
import torch
torch.cuda.device_count()

1

In [7]:
torch.device("cuda" if torch.cuda.is_available() else "cpu")

device(type='cuda')

In [8]:
# https://huggingface.co/bigscience/bloom-1b7

In [9]:
from transformers import AutoTokenizer, AutoModelForCausalLM

In [9]:
# from transformers import BloomTokenizerFast

In [10]:
# tokenizer = BloomTokenizerFast.from_pretrained(
#     "bigscience/bloom-1b7",
#     local_files_only=True
# )

In [10]:
model_name = "FlagAlpha/Llama2-Chinese-7b-Chat"

In [11]:
tokenizer = AutoTokenizer.from_pretrained(pretrained_model_name_or_path=model_name,
                                          local_files_only=True
                                          # use_auth_token=True,
                                         )

In [12]:
tokenizer.eos_token_id

2

In [13]:
tokenizer.pad_token_id

In [14]:
model = AutoModelForCausalLM.from_pretrained(model_name,
                                             local_files_only=True,
                                             device_map='auto',
                                             # torch_dtype=torch.float16,
                                             temperature=0.2, # must be strictly positive float
                                             do_sample=True,
                                             # use_auth_token=True,
                                            #  load_in_8bit=True,
                                            #  load_in_4bit=True
                                             )

2023-08-31 13:49:44.475718: I tensorflow/core/platform/cpu_feature_guard.cc:182] This TensorFlow binary is optimized to use available CPU instructions in performance-critical operations.
To enable the following instructions: AVX2 AVX512F FMA, in other operations, rebuild TensorFlow with the appropriate compiler flags.
2023-08-31 13:49:45.340707: W tensorflow/compiler/tf2tensorrt/utils/py_utils.cc:38] TF-TRT Warning: Could not find TensorRT


Loading checkpoint shards:   0%|          | 0/2 [00:00<?, ?it/s]

/usr/local/lib/python3.10/site-packages/transformers/generation/configuration_utils.py:362: UserWarning: `do_sample` is set to `False`. However, `temperature` is set to `0.9` -- this flag is only used in sample-based generation modes. You should set `do_sample=True` or unset `temperature`. This was detected when initializing the generation config instance, which means the corresponding file may hold incorrect parameterization and should be fixed.
  warnings.warn(
/usr/local/lib/python3.10/site-packages/transformers/generation/configuration_utils.py:367: UserWarning: `do_sample` is set to `False`. However, `top_p` is set to `0.6` -- this flag is only used in sample-based generation modes. You should set `do_sample=True` or unset `top_p`. This was detected when initializing the generation config instance, which means the corresponding file may hold incorrect parameterization and should be fixed.
  warnings.warn(


In [15]:
!nvidia-smi

Thu Aug 31 13:49:57 2023       
+---------------------------------------------------------------------------------------+
| NVIDIA-SMI 535.54.03              Driver Version: 535.54.03    CUDA Version: 12.2     |
|-----------------------------------------+----------------------+----------------------+
| GPU  Name                 Persistence-M | Bus-Id        Disp.A | Volatile Uncorr. ECC |
| Fan  Temp   Perf          Pwr:Usage/Cap |         Memory-Usage | GPU-Util  Compute M. |
|                                         |                      |               MIG M. |
|=========================================+======================+======================|
|   0  Tesla T4                       Off | 00000000:AF:00.0 Off |                    0 |
| N/A   69C    P0              33W /  70W |  13943MiB / 15360MiB |      1%      Default |
|                                         |                      |                  N/A |
+-----------------------------------------+----------------------+--

In [21]:
# https://github.com/shibing624/text2vec/issues/116

In [30]:
from torch.hub import _get_torch_home

_get_torch_home()

'/root/.cache/torch'

In [37]:
from langchain.embeddings import HuggingFaceInstructEmbeddings

instructor_embeddings = HuggingFaceInstructEmbeddings(model_name="/root/.cache/torch/sentence_transformers/GanymedeNil_text2vec-large-chinese",
                                                      # local_files_only=True,
                                                      model_kwargs={"device": "cuda"})

No sentence-transformers model found with name /root/.cache/torch/sentence_transformers/GanymedeNil_text2vec-large-chinese. Creating a new one with MEAN pooling.


In [46]:
instructor_embeddings

HuggingFaceInstructEmbeddings(client=INSTRUCTOR(
  (0): Transformer({'max_seq_length': 512, 'do_lower_case': False}) with Transformer model: BertModel 
  (1): Pooling({'word_embedding_dimension': 1024, 'pooling_mode_cls_token': False, 'pooling_mode_mean_tokens': True, 'pooling_mode_max_tokens': False, 'pooling_mode_mean_sqrt_len_tokens': False})
), model_name='/root/.cache/torch/sentence_transformers/GanymedeNil_text2vec-large-chinese', cache_folder=None, model_kwargs={'device': 'cuda'}, encode_kwargs={}, embed_instruction='Represent the document for retrieval: ', query_instruction='Represent the question for retrieving supporting documents: ')

In [41]:
dir(HuggingFaceInstructEmbeddings)

['Config',
 '__abstractmethods__',
 '__annotations__',
 '__class__',
 '__class_vars__',
 '__config__',
 '__custom_root_type__',
 '__delattr__',
 '__dict__',
 '__dir__',
 '__doc__',
 '__eq__',
 '__exclude_fields__',
 '__fields__',
 '__fields_set__',
 '__format__',
 '__ge__',
 '__get_validators__',
 '__getattribute__',
 '__getstate__',
 '__gt__',
 '__hash__',
 '__include_fields__',
 '__init__',
 '__init_subclass__',
 '__iter__',
 '__json_encoder__',
 '__le__',
 '__lt__',
 '__module__',
 '__ne__',
 '__new__',
 '__post_root_validators__',
 '__pre_root_validators__',
 '__pretty__',
 '__private_attributes__',
 '__reduce__',
 '__reduce_ex__',
 '__repr__',
 '__repr_args__',
 '__repr_name__',
 '__repr_str__',
 '__rich_repr__',
 '__schema_cache__',
 '__setattr__',
 '__setstate__',
 '__signature__',
 '__sizeof__',
 '__slots__',
 '__str__',
 '__subclasshook__',
 '__try_update_forward_refs__',
 '__validators__',
 '__weakref__',
 '_abc_impl',
 '_calculate_keys',
 '_copy_and_set_values',
 '_decompose

In [47]:
instructor_embeddings.embed_query("你好嗎?")

OutOfMemoryError: CUDA out of memory. Tried to allocate 16.00 MiB (GPU 0; 14.58 GiB total capacity; 14.39 GiB already allocated; 11.56 MiB free; 14.40 GiB reserved in total by PyTorch) If reserved memory is >> allocated memory try setting max_split_size_mb to avoid fragmentation.  See documentation for Memory Management and PYTORCH_CUDA_ALLOC_CONF

In [38]:
!nvidia-smi

Thu Aug 31 14:03:46 2023       
+---------------------------------------------------------------------------------------+
| NVIDIA-SMI 535.54.03              Driver Version: 535.54.03    CUDA Version: 12.2     |
|-----------------------------------------+----------------------+----------------------+
| GPU  Name                 Persistence-M | Bus-Id        Disp.A | Volatile Uncorr. ECC |
| Fan  Temp   Perf          Pwr:Usage/Cap |         Memory-Usage | GPU-Util  Compute M. |
|                                         |                      |               MIG M. |
|=========================================+======================+======================|
|   0  Tesla T4                       Off | 00000000:AF:00.0 Off |                    0 |
| N/A   70C    P0              33W /  70W |  13943MiB / 15360MiB |      0%      Default |
|                                         |                      |                  N/A |
+-----------------------------------------+----------------------+--

In [ ]:
from langchain.vectorstores import Chroma
persist_directory = 'db_cn'

vectordb = Chroma(persist_directory=persist_directory,
                  embedding_function=instructor_embeddings)

retriever = vectordb.as_retriever(search_kwargs={"k": 3})

In [ ]:
from langchain import PromptTemplate
from langchain.prompts.chat import (
    ChatPromptTemplate,
    SystemMessagePromptTemplate,
    AIMessagePromptTemplate,
    HumanMessagePromptTemplate,
)

In [ ]:
# https://python.langchain.com/docs/modules/model_io/models/chat/prompts

In [ ]:
from transformers import pipeline

pipe = pipeline("text-generation",
                model=model,
                tokenizer= tokenizer,
                # return_full_text=True,
                # torch_dtype=torch.bfloat16,
                # device_map="auto",
                max_new_tokens = 512,
                do_sample=True,
                top_k=30,
                num_return_sequences=1,
                eos_token_id=tokenizer.eos_token_id
                )

In [ ]:
from langchain.llms import HuggingFacePipeline
llm = HuggingFacePipeline(pipeline=pipe)

In [13]:
llm("你好嗎?")

/usr/local/lib/python3.10/site-packages/transformers/generation/utils.py:1411: UserWarning: You have modified the pretrained model configuration to control generation. This is a deprecated strategy to control generation and will be removed soon, in a future version. Please use a generation configuration file (see https://huggingface.co/docs/transformers/main_classes/text_generation )
  warnings.warn(


' 我是来自中国的学生，我的名字是张三。Љубљана是斯洛文尼亚的首都，它是一个非常美丽的城市，拥有很多古老的建筑和文化遗产。我很想去这里学习和旅游。你能告诉我一些关于这里的信息吗？\n'

In [ ]:
from langchain.chains import RetrievalQA, RetrievalQAWithSourcesChain

In [ ]:
qa = RetrievalQA.from_chain_type(llm=llm,
                                 chain_type="stuff",
                                 retriever=retriever,
                                 return_source_documents=True)

In [ ]:
import json
import pprint

# 1

In [ ]:
question1 = "如何设定判定逻辑?"

In [ ]:
%%time
res1 = qa({"query": question1})

# 2

In [5]:
question2 = "在哪里可以设定机种?"

In [1]:
# %%time
# res2 = qa({"query": question2})

In [ ]:
print(json.dumps(res1, indent=4))

In [6]:
question3 = "平台可以自动核准保修申请吗?"

In [2]:
# %%time
# qa({"query": question3})

In [7]:
question4 = "自动核准后还需要进行单号提交吗?"

In [3]:
# %%time
# qa({"query": question4})

In [8]:
question5 = "怎么查看保修进度?"

In [4]:
# %%time
# qa({"query": question5})